In [1]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 5.4 MB/s eta 0:00:00


In [2]:
import os, os.path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import optuna
from torch.utils.data import DataLoader, TensorDataset

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
## Encodes the labels in one hot format
def toOneHot(X):

    z = np.max(X) + 1
    X_oh = np.eye(z)[X]

    return X_oh

In [6]:
# Load and label EMG files
def loadAndLabel(folder_path,w,p):

    # folder_path -> path of the folder that contains each subject subfolder
    # w -> window size
    # p -> moving average delays

    data = [name for name in os.walk(folder_path)]

    X = []
    Y = []

    # Iterates folders
    for i in range(1,len(data)):

        # Iterates files in each folder
        for j in range(0,len(data[i][2])):

            # loads EMG data
            emgData = pd.read_csv(os.path.join(data[i][0], data[i][2][j]), sep='\t', header=None).values

            # Performs segmentation in windows
            for k in range(0,emgData.shape[0] - w + 1,w):

                d = emgData[k:k+w,:]
                d = np.abs(d) # Absolute value of the EMG data
                # Moving average filter for each channel
                for n in range(d.shape[1]-1):
                    d[:,n] = np.convolve(d[:,n], np.ones(p)/p, mode='same')
                X.append(d)

                # Generate labels for each window
                if data[i][2][j].split(".")[0] == '0':
                    Y.append(0)
                elif data[i][2][j].split(".")[0] == '1':
                    Y.append(1)
                elif data[i][2][j].split(".")[0] == '2':
                    Y.append(2)
                elif data[i][2][j].split(".")[0] == '3':
                    Y.append(3)
                elif data[i][2][j].split(".")[0] == '4':
                    Y.append(4)

    X = np.array(X)
    X = X/np.max(X)
    Y = np.array(Y)
    Y = toOneHot(Y)

    return X, Y

In [7]:
path = "/content/drive/MyDrive/EMG hand gestures dataset"
X, Y = loadAndLabel(path,200,20) # Load and label EMG data

In [8]:
# Split intro training and validation
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y)

In [9]:
import random

def add_noise(data, noise_factor=0.05):
  noise = np.random.normal(0, data.std() * noise_factor, data.shape)
  return data + noise

# Apply during data loading or before creating your dataset
X_train = [add_noise(x) for x in X_train]

In [10]:
# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
Y_train_tensor = torch.tensor(Y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
Y_test_tensor = torch.tensor(Y_test, dtype=torch.float32)

# Create datasets and dataloaders
train_dataset = TensorDataset(X_train_tensor, Y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, Y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False)

<ipython-input-10-fbe7f43a7250>:2: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:254.)
  X_train_tensor = torch.tensor(X_train, dtype=torch.float32)


In [11]:
Y_train_tensor = torch.argmax(Y_train_tensor, dim=1)
Y_test_tensor = torch.argmax(Y_test_tensor, dim=1)

In [12]:
Y_train_tensor = Y_train_tensor.type(torch.FloatTensor)
Y_test_tensor = Y_test_tensor.type(torch.FloatTensor)

In [13]:
# Define the EMGModel class
class EMGModel(nn.Module):
    def __init__(self, dropout_rate=0.5, lstm_hidden_size=12): # Add dropout_rate and lstm_hidden_size as arguments with default values
        super(EMGModel, self).__init__()
        #self.fc1 = nn.Linear(800, 1)  # Update input size to 200 * channels (4)
        #self.relu1 = nn.ReLU()
        self.lstm = nn.LSTM(800, lstm_hidden_size, batch_first=True) # Use lstm_hidden_size here
        self.dropout = nn.Dropout(p=dropout_rate) # Use dropout_rate here
        #self.fc2 = nn.Linear(16, 32)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(lstm_hidden_size, 5)  # Output classes: 5, adjust input size based on lstm_hidden_size

    def forward(self, x):
        batch_size = x.shape[0]
        x = x.view(batch_size, -1)  # Reshape to (batch_size, 200*channels)
        #x = self.relu1(self.fc1(x))
        x, _ = self.lstm(x)  # Get the output from the LSTM layer
        x = self.dropout(x)
        #x = x[:, -1, :]  # Take the output from the last time step
        x = self.relu2(x)
        x = self.fc3(x)
        return x

In [14]:
def validate(model, test_loader, criterion):
    model.eval()  # Set the model to evaluation mode
    total_loss = 0
    correct = 0
    with torch.no_grad():  # Disable gradient calculation
        for data, target in test_loader:
            output = model(data)
            loss = criterion(output, target)
            total_loss += loss.item()
            pred = output.argmax(dim=1, keepdim=True)  # Get predicted class
            actual_class_indices = torch.argmax(target, dim=1)
            correct += pred.eq(actual_class_indices.view_as(pred)).sum().item()

    avg_loss = total_loss / len(test_loader)
    accuracy = 100. * correct / len(test_loader.dataset)
    return avg_loss, accuracy

In [42]:
# Instantiate the model, optimizer, and loss function
model = EMGModel()
optimizer = optim.Adam(model.parameters(), lr=0.01,weight_decay=0.001)
criterion = nn.CrossEntropyLoss()
train_losses = []
val_losses = []
# Training loop
for epoch in range(200):
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()  # Reset gradients
        output = model(data)  # Forward pass
        loss = criterion(output, target) # Calculate loss, convert target to long
        loss.backward()  # Backpropagate
        optimizer.step()  # Update weights

    # Validation

    val_loss, val_accuracy = validate(model, test_loader, criterion)
    print(f"Epoch {epoch}: Val Loss = {val_loss:.4f}, Val Accuracy = {val_accuracy:.2f}%")

    # Append losses
    train_losses.append(loss.item()) # Append training loss for current batch
    val_losses.append(val_loss)

# Add EMGModel to safe globals
torch.serialization.add_safe_globals([EMGModel])

Epoch 0: Val Loss = 1.5045, Val Accuracy = 50.79%
Epoch 1: Val Loss = 1.2198, Val Accuracy = 61.54%
Epoch 2: Val Loss = 0.9944, Val Accuracy = 74.60%
Epoch 3: Val Loss = 0.8455, Val Accuracy = 79.12%
Epoch 4: Val Loss = 0.7401, Val Accuracy = 77.29%
Epoch 5: Val Loss = 0.6895, Val Accuracy = 81.81%
Epoch 6: Val Loss = 0.6436, Val Accuracy = 78.27%
Epoch 7: Val Loss = 0.6010, Val Accuracy = 82.30%
Epoch 8: Val Loss = 0.5695, Val Accuracy = 81.32%
Epoch 9: Val Loss = 0.5496, Val Accuracy = 84.13%
Epoch 10: Val Loss = 0.5230, Val Accuracy = 84.98%
Epoch 11: Val Loss = 0.5066, Val Accuracy = 84.00%
Epoch 12: Val Loss = 0.4998, Val Accuracy = 85.71%
Epoch 13: Val Loss = 0.4815, Val Accuracy = 86.08%
Epoch 14: Val Loss = 0.4614, Val Accuracy = 86.81%
Epoch 15: Val Loss = 0.4648, Val Accuracy = 85.59%
Epoch 16: Val Loss = 0.4663, Val Accuracy = 85.84%
Epoch 17: Val Loss = 0.4439, Val Accuracy = 85.59%
Epoch 18: Val Loss = 0.4426, Val Accuracy = 86.32%
Epoch 19: Val Loss = 0.4312, Val Accuracy

In [ ]:
import matplotlib.pyplot as plt

plt.plot(train_losses, label='Training Loss')
plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [52]:
def objective(trial):
    # Hyperparameters to optimize
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-1, log=True)
    batch_size = trial.suggest_int("batch_size", 32, 256, step=32)
    dropout_rate = trial.suggest_float("dropout_rate", 0.0, 0.5, step=0.1)
    lstm_hidden_size = trial.suggest_int("lstm_hidden_size", 8, 64, step=8)

    # Create datasets and dataloaders with the suggested batch size
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    # Create the model with suggested hyperparameters
    model = EMGModel(dropout_rate=dropout_rate, lstm_hidden_size=lstm_hidden_size)

    # Define optimizer and loss function
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.CrossEntropyLoss()

    # Training loop (you can reuse your existing training loop here)
    for epoch in range(200):  # Or your desired number of epochs
        for batch_idx, (data, target) in enumerate(train_loader):
            # ... (your training logic) ...
            optimizer.zero_grad()  # Reset gradients
            output = model(data)  # Forward pass
            loss = criterion(output, target) # Calculate loss, convert target to long
            loss.backward()  # Backpropagate
            optimizer.step()  # Update weights


        val_loss, val_accuracy = validate(model, test_loader, criterion)
        print(f"Epoch {epoch}: Val Loss = {val_loss:.4f}, Val Accuracy = {val_accuracy:.2f}%")

    return val_accuracy  # Return the validation accuracy for Optuna to optimize

In [55]:
# Create an Optuna study
study = optuna.create_study(direction="maximize")  # Maximize validation accuracy

# Optimize the objective function
study.optimize(objective, n_trials=20)  # Run for 100 trials

# Get the best hyperparameters
best_params = study.best_params
print(f"Best hyperparameters: {best_params}")

[I 2025-04-14 22:05:51,947] A new study created in memory with name: no-name-3f43d57d-cf79-42c3-9176-f86e1bd1bd1c


Epoch 0: Val Loss = 0.6508, Val Accuracy = 79.85%
Epoch 1: Val Loss = 0.4427, Val Accuracy = 84.25%
Epoch 2: Val Loss = 0.4089, Val Accuracy = 82.17%
Epoch 3: Val Loss = 0.4195, Val Accuracy = 85.10%
Epoch 4: Val Loss = 0.3484, Val Accuracy = 86.57%
Epoch 5: Val Loss = 0.3886, Val Accuracy = 87.18%
Epoch 6: Val Loss = 0.3887, Val Accuracy = 84.00%
Epoch 7: Val Loss = 0.3190, Val Accuracy = 88.28%
Epoch 8: Val Loss = 0.3453, Val Accuracy = 87.67%
Epoch 9: Val Loss = 0.3674, Val Accuracy = 87.42%
Epoch 10: Val Loss = 0.3260, Val Accuracy = 87.79%
Epoch 11: Val Loss = 0.3176, Val Accuracy = 87.79%
Epoch 12: Val Loss = 0.3031, Val Accuracy = 88.64%
Epoch 13: Val Loss = 0.3521, Val Accuracy = 85.84%
Epoch 14: Val Loss = 0.3224, Val Accuracy = 88.89%
Epoch 15: Val Loss = 0.3297, Val Accuracy = 88.40%
Epoch 16: Val Loss = 0.3372, Val Accuracy = 87.06%
Epoch 17: Val Loss = 0.3428, Val Accuracy = 87.91%
Epoch 18: Val Loss = 0.3172, Val Accuracy = 87.42%
Epoch 19: Val Loss = 0.3183, Val Accuracy

[I 2025-04-14 22:06:46,083] Trial 0 finished with value: 85.22588522588522 and parameters: {'learning_rate': 0.06005622962583965, 'batch_size': 96, 'dropout_rate': 0.5, 'lstm_hidden_size': 16}. Best is trial 0 with value: 85.22588522588522.


Epoch 199: Val Loss = 0.9368, Val Accuracy = 85.23%
Epoch 0: Val Loss = 1.5963, Val Accuracy = 20.02%
Epoch 1: Val Loss = 1.5698, Val Accuracy = 20.02%
Epoch 2: Val Loss = 1.5188, Val Accuracy = 30.77%
Epoch 3: Val Loss = 1.4355, Val Accuracy = 45.18%
Epoch 4: Val Loss = 1.3553, Val Accuracy = 51.40%
Epoch 5: Val Loss = 1.2884, Val Accuracy = 56.41%
Epoch 6: Val Loss = 1.2329, Val Accuracy = 62.15%
Epoch 7: Val Loss = 1.1864, Val Accuracy = 64.47%
Epoch 8: Val Loss = 1.1456, Val Accuracy = 67.16%
Epoch 9: Val Loss = 1.1096, Val Accuracy = 68.86%
Epoch 10: Val Loss = 1.0755, Val Accuracy = 72.89%
Epoch 11: Val Loss = 1.0408, Val Accuracy = 75.09%
Epoch 12: Val Loss = 1.0099, Val Accuracy = 76.56%
Epoch 13: Val Loss = 0.9787, Val Accuracy = 78.14%
Epoch 14: Val Loss = 0.9502, Val Accuracy = 78.88%
Epoch 15: Val Loss = 0.9239, Val Accuracy = 79.61%
Epoch 16: Val Loss = 0.8979, Val Accuracy = 80.10%
Epoch 17: Val Loss = 0.8744, Val Accuracy = 80.22%
Epoch 18: Val Loss = 0.8529, Val Accurac

[I 2025-04-14 22:07:25,147] Trial 1 finished with value: 87.91208791208791 and parameters: {'learning_rate': 0.0002613476987857788, 'batch_size': 128, 'dropout_rate': 0.30000000000000004, 'lstm_hidden_size': 64}. Best is trial 1 with value: 87.91208791208791.


...

Epoch 198: Val Loss = 0.6058, Val Accuracy = 87.55%
Epoch 199: Val Loss = 0.6226, Val Accuracy = 87.91%
Epoch 0: Val Loss = 1.6110, Val Accuracy = 20.02%
Epoch 1: Val Loss = 1.6081, Val Accuracy = 20.02%
Epoch 2: Val Loss = 1.6054, Val Accuracy = 20.02%
Epoch 3: Val Loss = 1.6027, Val Accuracy = 20.02%
Epoch 4: Val Loss = 1.5994, Val Accuracy = 20.02%
Epoch 5: Val Loss = 1.5954, Val Accuracy = 20.02%
Epoch 6: Val Loss = 1.5905, Val Accuracy = 20.02%
Epoch 7: Val Loss = 1.5842, Val Accuracy = 20.27%
Epoch 8: Val Loss = 1.5764, Val Accuracy = 21.00%
Epoch 9: Val Loss = 1.5663, Val Accuracy = 23.93%
Epoch 10: Val Loss = 1.5542, Val Accuracy = 42.00%
Epoch 11: Val Loss = 1.5397, Val Accuracy = 53.36%
Epoch 12: Val Loss = 1.5245, Val Accuracy = 65.45%
Epoch 13: Val Loss = 1.5079, Val Accuracy = 64.59%
Epoch 14: Val Loss = 1.4912, Val Accuracy = 63.37%
Epoch 15: Val Loss = 1.4740, Val Accuracy = 61.54%
Epoch 16: Val Loss = 1.4576, Val Accuracy = 67.52%
Epoch 17: Val Loss = 1.4398, Val Accura

[I 2025-04-14 22:21:32,689] Trial 18 finished with value: 86.56898656898657 and parameters: {'learning_rate': 0.00010129540270948826, 'batch_size': 256, 'dropout_rate': 0.1, 'lstm_hidden_size': 64}. Best is trial 3 with value: 89.98778998778998.


Epoch 199: Val Loss = 0.4892, Val Accuracy = 86.57%
Epoch 0: Val Loss = 1.5101, Val Accuracy = 21.00%
Epoch 1: Val Loss = 1.3242, Val Accuracy = 63.86%
Epoch 2: Val Loss = 1.1613, Val Accuracy = 68.99%
Epoch 3: Val Loss = 1.0309, Val Accuracy = 74.36%
Epoch 4: Val Loss = 0.9227, Val Accuracy = 79.00%
Epoch 5: Val Loss = 0.8391, Val Accuracy = 81.68%
Epoch 6: Val Loss = 0.7696, Val Accuracy = 82.54%
Epoch 7: Val Loss = 0.7136, Val Accuracy = 83.03%
Epoch 8: Val Loss = 0.6689, Val Accuracy = 83.76%
Epoch 9: Val Loss = 0.6338, Val Accuracy = 84.86%
Epoch 10: Val Loss = 0.6017, Val Accuracy = 84.86%
Epoch 11: Val Loss = 0.5754, Val Accuracy = 84.98%
Epoch 12: Val Loss = 0.5554, Val Accuracy = 85.47%
Epoch 13: Val Loss = 0.5361, Val Accuracy = 84.86%
Epoch 14: Val Loss = 0.5199, Val Accuracy = 84.86%
Epoch 15: Val Loss = 0.5045, Val Accuracy = 86.81%
Epoch 16: Val Loss = 0.4899, Val Accuracy = 86.57%
Epoch 17: Val Loss = 0.4771, Val Accuracy = 85.84%
Epoch 18: Val Loss = 0.4685, Val Accurac

[I 2025-04-14 22:21:58,926] Trial 19 finished with value: 88.4004884004884 and parameters: {'learning_rate': 0.0014491140565807993, 'batch_size': 160, 'dropout_rate': 0.30000000000000004, 'lstm_hidden_size': 32}. Best is trial 3 with value: 89.98778998778998.


Epoch 199: Val Loss = 0.2763, Val Accuracy = 88.40%
Best hyperparameters: {'learning_rate': 0.0009473497146793975, 'batch_size': 96, 'dropout_rate': 0.30000000000000004, 'lstm_hidden_size': 64}


In [56]:
# Create your final model with the best hyperparameters
final_model = EMGModel(dropout_rate=best_params["dropout_rate"],
                      lstm_hidden_size=best_params["lstm_hidden_size"])

# Define optimizer and loss function using best learning rate
optimizer = optim.Adam(final_model.parameters(), lr=best_params["learning_rate"])
criterion = nn.CrossEntropyLoss()

# Train the final model using your existing training loop and the best batch size
train_loader = DataLoader(train_dataset, batch_size=best_params["batch_size"], shuffle=True)

for epoch in range(200):  # Or your desired number of epochs
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()  # Reset gradients
        output = final_model(data)  # Forward pass
        loss = criterion(output, target) # Calculate loss, convert target to long
        loss.backward()  # Backpropagate
        optimizer.step()

        # Validation
    val_loss, val_accuracy = validate(final_model, test_loader, criterion)
    print(f"Epoch {epoch}: Val Loss = {val_loss:.4f}, Val Accuracy = {val_accuracy:.2f}%")

    # Append losses
    train_losses.append(loss.item()) # Append training loss for current batch
    val_losses.append(val_loss)

# Add EMGModel to safe globals
torch.serialization.add_safe_globals([EMGModel])

Epoch 0: Val Loss = 1.4972, Val Accuracy = 39.44%
Epoch 1: Val Loss = 1.1821, Val Accuracy = 69.72%
Epoch 2: Val Loss = 0.9629, Val Accuracy = 76.19%
Epoch 3: Val Loss = 0.8195, Val Accuracy = 81.20%
Epoch 4: Val Loss = 0.7170, Val Accuracy = 80.22%
Epoch 5: Val Loss = 0.6515, Val Accuracy = 84.13%
Epoch 6: Val Loss = 0.5940, Val Accuracy = 83.64%
Epoch 7: Val Loss = 0.5575, Val Accuracy = 84.13%
Epoch 8: Val Loss = 0.5333, Val Accuracy = 84.25%
Epoch 9: Val Loss = 0.4998, Val Accuracy = 84.13%
Epoch 10: Val Loss = 0.4853, Val Accuracy = 84.25%
Epoch 11: Val Loss = 0.4656, Val Accuracy = 86.57%
Epoch 12: Val Loss = 0.4464, Val Accuracy = 86.32%
Epoch 13: Val Loss = 0.4278, Val Accuracy = 87.42%
Epoch 14: Val Loss = 0.4268, Val Accuracy = 86.20%
Epoch 15: Val Loss = 0.4110, Val Accuracy = 86.45%
Epoch 16: Val Loss = 0.4045, Val Accuracy = 86.20%
Epoch 17: Val Loss = 0.3926, Val Accuracy = 86.94%
Epoch 18: Val Loss = 0.3914, Val Accuracy = 85.71%
Epoch 19: Val Loss = 0.3824, Val Accuracy

In [58]:
torch.save(final_model, '/content/drive/MyDrive/EMG hand gestures dataset/model.pth')

In [15]:
def test_model(model, test_loader):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0
    with torch.no_grad():  # Disable gradient calculation
        for data, target in test_loader:
            output = model(data)
            _, predicted = torch.max(output.data, 1)
            total += target.size(0)
            correct += (predicted == torch.argmax(target, dim=1)).sum().item()

    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy:.2f}%")

In [16]:
# Explicitly add safe globals before loading the model.
torch.serialization.add_safe_globals([EMGModel, nn.Linear, nn.ReLU, nn.LSTM, nn.Dropout])

# Load the saved model
final_model = torch.load('/content/drive/MyDrive/EMG hand gestures dataset/model.pth')

In [17]:
test_model(final_model, test_loader)

Test Accuracy: 97.07%
